reading the files

In [107]:
import pandas as pd 

In [108]:
manufacturer_df = pd.read_pickle('cumulative_manufacturer.pkl')
manufacturer_df.head()

,manufacture_id,year,month,region,city_lat,city_long,city,state,country,quantity
0,1,2021,1,West,33.1447,117.1658,San Marcos,CA,US,75
1,2,2020,6,Unknown,0.0000,0.0000,Unknown,Unknown,Unknown,12
2,3,2021,7,Unknown,37.5665,126.9780,Seoul,Unknown,South Korea,4
3,4,2018,11,West,37.4419,122.1430,Palo Alto,CA,US,11
4,5,2019,5,West,33.1959,117.3795,Oceanside,CA,US,1


In [109]:
ev_car_df = pd.read_pickle('cumulative_df.pkl')
ev_car_df.head()

,year,month,region,city_lat,city_long,city,state,country,vehicle_pop_quantity,ev_cs_quantity
0,0,0,Midwest,29.093133,-97.315846,Dewitt,MI,US,0.0,1.0
1,0,0,Midwest,31.997366,-102.077948,Midland,MI,US,0.0,5.0
2,0,0,Midwest,34.341053,-82.950328,Hart,MI,US,0.0,1.0
3,0,0,Midwest,38.577359,-92.172427,Jefferson City,MO,US,0.0,2.0
4,0,0,Midwest,38.647738,-88.923211,Marion,IL,US,0.0,1.0


creating a new data frame 

In [110]:
cumulative_df = pd.merge(ev_car_df,manufacturer_df, how='outer', on= ['year','month',	'region'	,'city_lat','city_long','city',	'state'	,'country'])

In [111]:
cumulative_df.head(5)

,year,month,region,city_lat,city_long,city,state,country,vehicle_pop_quantity,ev_cs_quantity,manufacture_id,quantity
0,0,0,Midwest,29.093133,-97.315846,Dewitt,MI,US,0.0,1.0,NaN,NaN
1,0,0,Midwest,31.997366,-102.077948,Midland,MI,US,0.0,5.0,NaN,NaN
2,0,0,Midwest,34.341053,-82.950328,Hart,MI,US,0.0,1.0,NaN,NaN
3,0,0,Midwest,38.577359,-92.172427,Jefferson City,MO,US,0.0,2.0,NaN,NaN
4,0,0,Midwest,38.647738,-88.923211,Marion,IL,US,0.0,1.0,NaN,NaN


In [112]:
cumulative_df['vehicle_pop_quantity'] = cumulative_df['vehicle_pop_quantity'].fillna(0).astype(int)
cumulative_df['manufacture_id'] = cumulative_df['manufacture_id'].fillna(0).astype(int)
cumulative_df['quantity'] = cumulative_df['quantity'].fillna(0).astype(int)

In [113]:
cumulative_df.head(5)

,year,month,region,city_lat,city_long,city,state,country,vehicle_pop_quantity,ev_cs_quantity,manufacture_id,quantity
0,0,0,Midwest,29.093133,-97.315846,Dewitt,MI,US,0,1.0,0,0
1,0,0,Midwest,31.997366,-102.077948,Midland,MI,US,0,5.0,0,0
2,0,0,Midwest,34.341053,-82.950328,Hart,MI,US,0,1.0,0,0
3,0,0,Midwest,38.577359,-92.172427,Jefferson City,MO,US,0,2.0,0,0
4,0,0,Midwest,38.647738,-88.923211,Marion,IL,US,0,1.0,0,0


spliting into the tables

we need to add only manufacturer 0 as a null value 

In [114]:
manufacturer_dim = cumulative_df[['manufacture_id']].drop_duplicates(keep='first')
manufacturer_dim

,manufacture_id
0,0
18582,1
18583,2
18584,3
18585,4
18586,5
18587,6
18588,7
18589,8
18590,9


we can try to see if we can get location id from the manufacturer table and create only new location id for ev charging facts
first check to see if adding location id will cause increase in the df

In [115]:
cumulative_df.columns

Index(['year', 'month', 'region', 'city_lat', 'city_long', 'city', 'state',
       'country', 'vehicle_pop_quantity', 'ev_cs_quantity', 'manufacture_id',
       'quantity'],
      dtype='object')

In [116]:
len(cumulative_df)

18608

In [117]:
location_df = cumulative_df[[ 'city_lat', 'city_long', 'region','city', 'state',
       'country']].drop_duplicates(keep='first')
location_df.head()

,city_lat,city_long,region,city,state,country
0,29.093133,-97.315846,Midwest,Dewitt,MI,US
1,31.997366,-102.077948,Midwest,Midland,MI,US
2,34.341053,-82.950328,Midwest,Hart,MI,US
3,38.577359,-92.172427,Midwest,Jefferson City,MO,US
4,38.647738,-88.923211,Midwest,Marion,IL,US


In [118]:
len(location_df)

4668

In [119]:
import numpy as np
location_df['lat_prev'] =float(0)
location_df['long_prev'] = float(0)
# convert NaN values to null dates
location_df['updated_at'] = pd.to_datetime(np.NaN, errors='coerce')
location_df['latitude'] = location_df['city_lat']
location_df['longitude'] = location_df['city_long']

In [120]:
location_df =location_df.reset_index(drop=True)
location_df['location_id'] =  location_df.index+1000

In [121]:
location_df.head()

,city_lat,city_long,region,city,state,country,lat_prev,long_prev,updated_at,latitude,longitude,location_id
0,29.093133,-97.315846,Midwest,Dewitt,MI,US,0.0,0.0,NaT,29.093133,-97.315846,1000
1,31.997366,-102.077948,Midwest,Midland,MI,US,0.0,0.0,NaT,31.997366,-102.077948,1001
2,34.341053,-82.950328,Midwest,Hart,MI,US,0.0,0.0,NaT,34.341053,-82.950328,1002
3,38.577359,-92.172427,Midwest,Jefferson City,MO,US,0.0,0.0,NaT,38.577359,-92.172427,1003
4,38.647738,-88.923211,Midwest,Marion,IL,US,0.0,0.0,NaT,38.647738,-88.923211,1004


In [122]:
location_df.columns

Index(['city_lat', 'city_long', 'region', 'city', 'state', 'country',
       'lat_prev', 'long_prev', 'updated_at', 'latitude', 'longitude',
       'location_id'],
      dtype='object')

In [123]:
cumulative_df_location= pd.merge(cumulative_df, location_df, how='outer', on=['city_lat', 'city_long', 'region', 'city',
       'state', 'country'])

date dim 

In [124]:
cumulative_df.columns

Index(['year', 'month', 'region', 'city_lat', 'city_long', 'city', 'state',
       'country', 'vehicle_pop_quantity', 'ev_cs_quantity', 'manufacture_id',
       'quantity'],
      dtype='object')

In [125]:
# # fill missing values with a default year value of 2020
# default_year = 1979
# df = cumulative_df.fillna({'year': default_year})
# replace 0 month values with 1 and year values with 1905
cumulative_df.loc[cumulative_df['month'] == 0, 'month'] = 1
cumulative_df.loc[cumulative_df['year'] == 0, 'year'] = 1905




In [126]:
month_dim = cumulative_df[['month','year']].drop_duplicates(keep='first')
month_dim = month_dim.reset_index(drop=True)
month_dim

,month,year
0,1,1905
1,8,1995
2,10,1996
3,1,1997
4,7,1997
...,...,...
173,1,2023
174,2,2023
175,8,2022
176,6,2022


In [127]:
# add quarter column
# month_dim['quarter'] = pd.PeriodIndex(pd.to_datetime(month_dim['year'].astype(str) + month_dim['month'].astype(str), format='%Y%m'), freq='Q')
# convert month and year columns to a datetime object
date = pd.to_datetime(month_dim['year'].astype(str) + '-' + month_dim['month'].astype(str) + '-01', format='%Y-%m-%d')

# add quarter column to the dataframe
month_dim['quarter'] = date.dt.quarter
month_dim = month_dim.reset_index(drop=True)
month_dim

,month,year,quarter
0,1,1905,1
1,8,1995,3
2,10,1996,4
3,1,1997,1
4,7,1997,3
...,...,...,...
173,1,2023,1
174,2,2023,1
175,8,2022,3
176,6,2022,2


In [128]:
month_dim.head()

,month,year,quarter
0,1,1905,1
1,8,1995,3
2,10,1996,4
3,1,1997,1
4,7,1997,3


adding index 

In [129]:
month_dim =month_dim.reset_index(drop=True)
month_dim['month_id'] =  month_dim.index+1

In [130]:
month_dim.head()

,month,year,quarter,month_id
0,1,1905,1,1
1,8,1995,3,2
2,10,1996,4,3
3,1,1997,1,4
4,7,1997,3,5


In [131]:
len(cumulative_df)

18608

In [132]:
cumulative_df_date= pd.merge(cumulative_df_location, month_dim, how='outer', on=['month', 'year'] )

----------------------------------------------------- -                        ----------------------------------------------------
----------------------------------------------------- creating and loading tables  ----------------------------------------------------
----------------------------------------------------- -                        ----------------------------------------------------